# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [1]:
ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1

192.168.56.188


In [2]:
IP_LEADER=$(ip a s eth0 | grep 'inet\s' | awk '{ print $2}' | cut -d '/' -f 1)
echo $IP_LEADER

192.168.56.188


In [3]:
docker node ls || docker swarm init --advertise-addr ${IP_LEADER}

Error response from daemon: This node is not a swarm manager. Use "docker swarm init" or "docker swarm join" to connect this node to swarm and try again.
Swarm initialized: current node (moj06efgjb0htcgv1vdru47l9) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-56qhww46mz9dbgt43mlq2ce8cib94xcqq87pfg3e0v0l46n7ap-c44lijzts7b1scfnoom351cs5 192.168.56.188:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



In [4]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

SWMTKN-1-56qhww46mz9dbgt43mlq2ce8cib94xcqq87pfg3e0v0l46n7ap-c44lijzts7b1scfnoom351cs5


## SSH einrichten

In [5]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

/home/pi/.ssh/id_rsa


Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [6]:
ping -c 1 cluster-01.local | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1

192.168.56.189


In [7]:
for i in 01 02 03 04; do
    # Remove host key for ip
    CL_HOSTNAME=cluster-${i}
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    ssh-keygen -R ${CL_HOSTNAME}.local
    # IP Cluster 
    ssh-keygen -R ${CL_IP}
    # Add ip into known_hosts
    ssh-keyscan -H ${CL_HOSTNAME}.local >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_HOSTNAME} >> ~/.ssh/known_hosts
    ssh-keyscan -H ${CL_IP} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@${CL_HOSTNAME}.local
done

cluster-01
# Host cluster-01 found: line 4 type RSA
# Host cluster-01 found: line 5 type ECDSA
# Host cluster-01 found: line 6 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host cluster-01.local found: line 1 type ECDSA
# Host cluster-01.local found: line 2 type RSA
# Host cluster-01.local found: line 3 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# Host 192.168.56.189 found: line 1 type RSA
# Host 192.168.56.189 found: line 2 type ECDSA
# Host 192.168.56.189 found: line 3 type ED25519
/home/pi/.ssh/known_hosts updated.
Original contents retained as /home/pi/.ssh/known_hosts.old
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01.local SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cluster-01 SSH-2.0-OpenSSH_6.7p1 Raspbian-5+deb8u3
# cl

Alle Clients als Worker dem Swarm beitreten.

In [8]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}.local
    echo $CL_HOSTNAME
    CL_IP=$(ping -c 1 ${CL_HOSTNAME} | grep icmp | cut -d '(' -f 2 | cut -d')' -f 1)
    echo $CL_IP
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP_LEADER}:2377" || \
    ssh pi@cluster-${i} "docker swarm leave --force" && \
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP_LEADER}:2377"
done

cluster-01.local
192.168.56.189
This node joined a swarm as a worker.
cluster-02.local
192.168.56.191
This node joined a swarm as a worker.
cluster-03.local
192.168.56.187
This node joined a swarm as a worker.
cluster-04.local
192.168.56.196
This node joined a swarm as a worker.


In [9]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
moj06efgjb0htcgv1vdru47l9 *   cluster-00          Ready               Active              Leader
kkhtio591ofuaa00nnfxhym65     cluster-01          Ready               Active              
lelrz13rw5nfzgmkq3yt17elz     cluster-02          Ready               Active              
htuzsrxojzhmkutmv4e81h1nx     cluster-03          Ready               Active              
f50cw2jio2dcqnreeg2nuqlez     cluster-04          Ready               Active              


## Visualizer anschauen

Einen graphischen Output als Service starten.

In [10]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  alexellis2/visualizer-arm

nwaqw02ep704235atyb5zpqsc

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [11]:
echo "http://${IP_LEADER}:8000"

http://192.168.56.188:8000


In [12]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [13]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  --no-resolve-image \
  --detach=false \
  stefanscherer/monitor:1.1.0

h0eg9204yai7a2pfsjm3uzz7g

all progress: 0 out of 5 tasks 
z13rw5nf: new       
srxojzhm: new       
w2jio2dc: new       
6efgjb0h: new       
all progress: 5 out of 5 tasks 
fy: Service converged to verify that tasks are stable... 

Es läuft noch kein whoami.

In [14]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [15]:
docker service create --name whoami \
  --no-resolve-image \
  --detach=false \
  stefanscherer/whoami:1.1.0 

3hd0ocoqhk0xwpop4ju5w0oq8

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
fy: Service converged to verify that tasks are stable... 

In [16]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          1/1                 stefanscherer/whoami:1.1.0         


In [17]:
docker service scale --detach=false whoami=5

whoami scaled to 5

all progress: 0 out of 5 tasks 
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
   K$<3>$<3>
all progress: 5 out of 5 tasks 
fy: Service converged to verify that tasks are stable... 

In [18]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          5/5                 stefanscherer/whoami:1.1.0         


In [19]:
docker service scale --detach=false whoami=15

whoami scaled to 15

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
fy: Service converged to verify that tasks are stable... 

In [20]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          15/15               stefanscherer/whoami:1.1.0         


In [21]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [22]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [23]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [24]:
docker service scale --detach=false whoami=1

whoami scaled to 1

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks  
fy: Service converged to verify that tasks are stable... 

In [25]:
docker service scale --detach=false whoami=40

whoami scaled to 40

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [26]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [27]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.1.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [28]:
docker service update --update-parallelism 5 \
  --detach=false \
  --image stefanscherer/whoami:1.2.0 whoami

whoami

all progress: 40 out of 40 tasks 
fy: Service converged to verify that tasks are stable... 

In [29]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
h0eg9204yai7        monitor             global              5/5                 stefanscherer/monitor:1.1.0        
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp
3hd0ocoqhk0x        whoami              replicated          40/40               stefanscherer/whoami:1.2.0         


In [30]:
END=10
for i in $(seq 1 $END); do
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    docker service scale --detach=false whoami=1 
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=15
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
    docker service scale --detach=false whoami=40
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service update --update-parallelism 5 \
        --detach=false \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 5
done

whoami

all progress: 40 out of 40 tasks 
whoami scaled to 1ged to verify that tasks are stable... 

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks  
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoami scaled to 15ed to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $

:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ed to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoamirvice converged to

:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 0 out of 15 tasks 
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
:   $<3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
5:   <3>$<3>
ll progress: 15 out of 15 tasks 
whoami scaled to 40ed to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoamirvice converged to verify that tasks are stable... 

all progress: 40 out of 40 tasks 
whoami scaled to 1ged to verify that tasks are stable... 

all progress: 0 out of 1 tasks 
all progress: 1 out of 1 tasks 
whoamirvice converged to verify that tasks are stable... 

al

# Aufräumen

In [31]:
docker service rm whoami

whoami


In [32]:
docker service rm monitor

monitor


In [33]:
docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
moj06efgjb0htcgv1vdru47l9 *   cluster-00          Ready               Active              Leader
kkhtio591ofuaa00nnfxhym65     cluster-01          Ready               Active              
lelrz13rw5nfzgmkq3yt17elz     cluster-02          Ready               Active              
htuzsrxojzhmkutmv4e81h1nx     cluster-03          Ready               Active              
f50cw2jio2dcqnreeg2nuqlez     cluster-04          Ready               Active              


In [34]:
docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                              PORTS
nwaqw02ep704        visualizer          replicated          1/1                 alexellis2/visualizer-arm:latest   *:8000->8080/tcp


In [35]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

Node left the swarm.
Node left the swarm.
Node left the swarm.
Node left the swarm.


In [36]:
docker service rm visualizer

visualizer


In [37]:
docker swarm leave --force

Node left the swarm.
